In [877]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
import pandas as pd
import numpy as np
import pygeohash as pgh
from sklearn import metrics, cluster

%matplotlib inline

In [878]:
data = 'data/unit_data.csv'
df = pd.read_csv(data)
df.head()

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,avg_price,avg_cpm,zip_code,zip_code_id,lat,lon,size,height,width,price,rate_card_price,floor_price,cpm,impressions,unit_type,media_type,supplier_id,supplier_face_id
0,203158,900.00,10.11,4031,30823,40.698358,-96.704913,"10' 6"" x 22' 9""",10.5,22.75,900.0000,1080.0,NaN,10.11,89016,billboard,Billboard,39,9663
1,114094,295.00,8.18,4016,37262,46.216275,-116.005442,10' x 20',10.0,20.00,295.0000,354.0,NaN,8.18,36068,billboard,Billboard,39,39202
2,469314,3133.33,7.75,4039,19949,39.116990,-84.439070,14' x 48',14.0,48.00,3500.0000,4200.0,NaN,8.66,404084,billboard,Billboard,39,2604
3,413568,812.58,4.29,4047,16341,36.408146,-86.419902,12x25,300.0,144.00,812.5765,1020.0,NaN,4.29,189548,bulletin,Billboard,60,9548
4,227209,750.00,3.35,4028,17440,32.304140,-90.255340,"10' 6"" x 22' 9""",10.5,22.75,750.0000,900.0,NaN,3.35,223728,billboard,Billboard,39,3016


In [879]:
df.dtypes

id                    int64
avg_price           float64
avg_cpm              object
zip_code              int64
zip_code_id           int64
lat                 float64
lon                 float64
size                 object
height              float64
width               float64
price               float64
rate_card_price     float64
floor_price         float64
cpm                 float64
impressions           int64
unit_type            object
media_type           object
supplier_id           int64
supplier_face_id     object
dtype: object

In [880]:
df.describe()

,id,avg_price,zip_code,zip_code_id,lat,lon,height,width,price,rate_card_price,floor_price,cpm,impressions,supplier_id
count,3.414000e+04,34140.000000,34140.000000,34140.000000,34140.000000,34140.000000,34140.000000,34140.000000,34140.000000,34115.000000,39.000000,34140.000000,3.414000e+04,34140.000000
mean,4.437189e+05,1393.770573,4030.562917,22548.048418,37.161701,-90.932532,31.994941,55.484715,1405.414986,1728.873665,12503.179487,7.385894,3.749847e+05,65.269479
std,7.112636e+05,2753.777827,16.102772,11394.030821,5.003686,12.402323,94.468999,118.866561,2729.621317,5191.813023,13669.002949,89.433617,8.500559e+05,141.019508
min,5.280800e+04,-6623.790000,4001.000000,10.000000,21.294597,-157.862990,2.000000,1.000000,100.000000,0.000000,0.000000,0.010000,0.000000e+00,28.000000
25%,1.333852e+05,550.000000,4017.000000,14042.000000,33.468421,-96.211025,10.500000,22.750000,550.000000,666.000000,4000.000000,2.330000,1.147110e+05,39.000000
50%,2.642960e+05,809.500000,4035.000000,20582.000000,36.578443,-88.097195,10.500000,22.750000,800.000000,960.000000,11000.000000,3.930000,2.162540e+05,39.000000
75%,4.760792e+05,1375.000000,4042.000000,32842.000000,41.091103,-81.773639,14.000000,48.000000,1400.000000,1650.000000,15000.000000,6.950000,4.044670e+05,39.000000
max,4.238355e+06,128750.000000,4056.000000,43307.000000,48.939110,-70.918949,4800.000000,10884.000000,93720.000000,667200.000000,68200.000000,15000.000000,1.000000e+08,1717.000000


In [881]:
# print(df.isnull().sum())
df.dropna(subset=['supplier_face_id', 'rate_card_price'], inplace=True)
# df.isnull().sum()

In [882]:
print(df.media_type.value_counts())
df = df[df.media_type == 'Billboard']

Billboard           32687
Street Furniture      749
Retail/Venues         351
Alternative           162
Wallscape             113
Transit                42
Wildposting             7
Windowscape             2
Name: media_type, dtype: int64


In [883]:
df.height.apply(lambda h: round(h / 5))
df.width.apply(lambda w: round(w / 5))
df['sqft'] = [l[0] * l[1] for l in list(zip(df.height, df.width))]
df['size_ratio'] = [l[1] / l[0] for l in list(zip(df.height, df.width))]
df['geo_hash'] = [pgh.encode(l[0], l[1], precision=2) for l in list(zip(df.lat, df.lon))]
# cols = ['id', 'supplier_id', 'sqft', 'geo_hash', 'price', 'cpm', 'impressions', 'media_type']
cols = ['id', 'supplier_id', 'sqft', 'size_ratio', 'geo_hash', 'price', 'cpm', 'impressions']
orig_df = df
df = df[cols]
df.head()

,id,supplier_id,sqft,size_ratio,geo_hash,price,cpm,impressions
0,203158,39,238.875,2.166667,9z,900.0000,10.11,89016
1,114094,39,200.000,2.000000,c2,295.0000,8.18,36068
2,469314,39,672.000,3.428571,dn,3500.0000,8.66,404084
3,413568,60,43200.000,0.480000,dn,812.5765,4.29,189548
4,227209,39,238.875,2.166667,9v,750.0000,3.35,223728


In [884]:
# print(df.geo_hash.nunique())
# print(df.media_type.nunique())

# df = pd.get_dummies(df, columns=['geo_hash', 'media_type'])
# df = pd.get_dummies(df, columns=['media_type'])


In [885]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# exclude geo_hash and media_type because they are dummies now
exclude = ['id', 'supplier_id', 'lat', 'lon', 'geo_hash', 'media_type']

feature_cols = list(df.columns.difference(exclude))
orig_cols = df.columns
# don't scale the dummies
df = pd.get_dummies(df, columns=['supplier_id'])
X = scaler.fit_transform(df[feature_cols])
print(X.shape)
# scale only the feature cols, then add the dummies and combine to an array
# first array is only the feature cols, second is only the supplier id dummies
new_x = pd.concat([pd.DataFrame(X, columns=feature_cols), df[df.columns.difference(exclude)]])
new_x.dropna(axis = 0, how ='any', inplace=True)
X = new_x.to_numpy()
print(X.shape)
new_x.isnull().sum()

(32687, 5)
(32687, 116)


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


cpm                0
impressions        0
price              0
size_ratio         0
sqft               0
                  ..
supplier_id_953    0
supplier_id_954    0
supplier_id_96     0
supplier_id_97     0
supplier_id_985    0
Length: 116, dtype: int64

In [886]:
nullseries = new_x.isnull().sum()
print(nullseries[nullseries < 0])
print(X.shape)
np.where(np.isnan(X))

Series([], dtype: int64)
(32687, 116)


(array([], dtype=int64), array([], dtype=int64))

In [ ]:
wcss = []
ss = []
iter_range = range(2, 16)
for i in iter_range:
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    labels = kmeans.labels_
    ss.append(metrics.silhouette_score(X, labels, metric='euclidean'))

In [ ]:
plt.plot(iter_range, wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
plt.plot(iter_range, ss)
plt.title('Cliff Method')
plt.xlabel('Number of clusters')
plt.ylabel('SS')
plt.show()

In [ ]:
from sklearn.cluster import KMeans
kmeans = cluster.KMeans(n_clusters=8, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(X)

labels = kmeans.labels_
centroids = kmeans.cluster_centers_
inertia = kmeans.inertia_
metrics.silhouette_score(X, labels, metric='euclidean')

In [ ]:
df['label'] = labels
# df['media_type'] = orig_df['media_type']
df.head()

In [ ]:
# below error means too much data
# ValueError: The number of observations must be larger than the number of variables.
# labeled_cols = ['supplier_id', 'sqft', 'geo_hash', 'price', 'cpm', 'impressions', 'media_type', 'label']
# convert media type to number
plot_cols = ['supplier_id', 'sqft', 'size_ratio', 'price', 'impressions']
# sns.pairplot(df[:1000], x_vars=plot_cols, y_vars= labeled_cols, hue='label')
print(df.columns)
df.head()
# sns.pairplot(df, x_vars=plot_cols, y_vars=plot_cols, hue='label');

In [ ]:
# df[['label', 'id', 'supplier_id', 'sqft', 'geo_hash', 'price', 'cpm', 'impressions', 'media_type']].head()
df['supplier_id'] = orig_df['supplier_id']
df[['label', 'id', 'supplier_id', 'sqft', 'size_ratio', 'geo_hash', 'price', 'cpm', 'impressions']].head()

In [ ]:
from sklearn.cluster import DBSCAN
print(feature_cols)
for min_sample in range(1, 20, 2):
    for eps in range(2, 10, 2):
        clustering = DBSCAN(eps=(eps/10), min_samples=min_sample).fit(df[feature_cols])
        if (np.unique(clustering.labels_)[0] != -1):
            df['cluster'] = clustering.labels_
            print(min_sample, eps)
            print(metrics.silhouette_score(df[feature_cols], df['cluster'], metric='euclidean'))

In [ ]:
clustering = DBSCAN(eps=.4, min_samples=1).fit(df[feature_cols])
metrics.silhouette_score(df[feature_cols], df['cluster'], metric='euclidean')

In [ ]:
plot_cols = ['supplier_id', 'sqft', 'size_ratio', 'price', 'impressions']
sns.pairplot(df, x_vars=plot_cols, y_vars=plot_cols, hue='label');

In [ ]:
df['lat'] = orig_df['lat']
df['lon'] = orig_df['lon']
df.head()

In [ ]:
# chooose target
y = df.loc[df['id'] == 469314]

In [ ]:
# find all rows with the same label and/or cluster
# calculate haversine distance
# sort and find closest
X = df[df['label'] == y['label'].values[0]]
X.describe()

In [ ]:
X.nunique()

In [ ]:
def geo_dist(lat1, lon1, lat2, lon2):
    x = np.absolute((float(lat1)**2) - (float(lat2)**2))
    y = np.absolute((float(lon1)**2) - (float(lon2)**2))
    coord_sum = x + y
    sqrt = np.sqrt(coord_sum)
    return sqrt

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [float(lon1), float(lat1), float(lon2), float(lat2)])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [ ]:
output_array = []
for index1, row1 in X.iterrows():
    for index2, row2 in X.iterrows():
        if row1[0] == row2[0]:
            pass
#         if index1 < 10:
#             print(row1['id'], y['id'].values[0])
        if row1['id'] != y['id'].values[0]:
            pass
        else:            
            h_dist = haversine(row1['lon'], row1['lat'], row2['lon'], row2['lat'])
#             h_dist = geo_dist(row1[1], row1[2], row2[1], row2[2])
            output_array.append([row1.id, row2.id, h_dist])

In [ ]:
distance_matrix_df = pd.DataFrame(output_array, columns=['id1', 'id2', 'haversine_distance'])
distance_matrix_df.head()

In [ ]:
distance_matrix_df.sort_values(by=['haversine_distance'], ascending=True, inplace=True)

In [ ]:
distance_matrix_df[:20]